## Taller Final - Corte 3: De los Datos Crudos al Insight Gerencial
### Sector: Industria Automotriz
#### Grupo 2: Juanita Merchán y Annie Bonilla

In [28]:
# Instalación de librerías necesarias
%pip install pandas openpyxl

# Asegurar que el pip esté actualizado
%pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [29]:
# Importar la librería pandas
import pandas as pd
import numpy as np

# Cargar el archivo tabla1_emc_general.xlsx
df_general = pd.read_excel('tabla1_emc_general.xlsx')

# Cargar el archivo tabla2_emc_vehiculos.xlsx
df_vehiculos = pd.read_excel('tabla2_emc_vehiculos.xlsx')

# Cargar el archivo tabla3_demografia_censo2018.xlsx
df_demografia = pd.read_excel('tabla3_demografia_censo2018.xlsx')

# Mostrar las primeras filas para verificar
print("Datos de tabla1_emc_general.xlsx:")
print(df_general.head())
print("\n")
print("\nDatos de tabla2_emc_vehiculos.xlsx:")
print(df_vehiculos.head())
print("\n")
print("\nDatos de tabla3_demografia_censo2018.xlsx:")
print(df_demografia.head())
print("\n")

Datos de tabla1_emc_general.xlsx:
  Unnamed: 0 Unnamed: 1                                          Antioquia  \
0        Año        Mes  Total comercio minorista y vehículos en el dep...   
1        NaN        NaN                                                NaN   
2        NaN        NaN                                                NaN   
3       2025      Enero                                         123.665347   
4        NaN    Febrero                                         120.511263   

                                          Unnamed: 3  \
0  Comercio, mantenimiento y reparación de vehícu...   
1                                                NaN   
2                 4511. Vehículos automotores nuevos   
3                                         108.626431   
4                                         126.738337   

                                          Unnamed: 4  \
0                                                NaN   
1                                               

#### Ingenieria de Datos con Python y Pandas

In [ ]:
#Estandariza y limpia los nombres de columnas de un DataFrame para evitar problemas al manipularlos.
def clean_column_names(df):
    df = df.copy()
    df.columns = (
        df.columns.str.strip()
                    .str.lower()
                    .str.replace(" ", "_")
                    .str.replace("\n", "_")
                    .str.replace("(", "")
                    .str.replace(")", "")
    )
    return df


def convert_numeric_columns(df, cols):
    df = df.copy()
    for c in cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")
    return df


def summarize_missing(df):
    missing = df.isnull().sum()
    pct = (missing / len(df)) * 100
    return pd.DataFrame({"missing": missing, "missing_pct": pct})


def fill_missing_median(df):
    df = df.copy()
    num_cols = df.select_dtypes(include="number").columns
    for c in num_cols:
        df[c] = df[c].fillna(df[c].median())
    return df


def drop_constant_columns(df):
    nunique = df.nunique()
    const_cols = nunique[nunique <= 1].index.tolist()
    return df.drop(columns=const_cols), const_cols


##### Base de datos 1: Informativos Comercio al por Menor - Por departamentos (EMC)

In [31]:
file_path = 'tabla1_emc_general.xlsx'

# Se lee el archivo Excel y se carga en un DataFrame de Pandas.
# El bloque 'try-except' es una buena práctica para manejar errores si el archivo no existe.
try:
    df1 = pd.read_excel(file_path)
    print("✅ Archivo 'tabla1_emc_general.xlsx' cargado exitosamente.")
except FileNotFoundError:
    print(f"❌ Error: No se encontró el archivo '{file_path}'. Asegúrate de que esté en la misma carpeta.")

# Mostramos las primeras 5 filas para tener una idea inicial de los datos.
print("\n--- Vista Previa de los Datos Crudos ('Sucios') ---")
df1.head()

✅ Archivo 'tabla1_emc_general.xlsx' cargado exitosamente.

--- Vista Previa de los Datos Crudos ('Sucios') ---


,Unnamed: 0,Unnamed: 1,Antioquia,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 54,Unnamed: 55,Otros departamentos,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63
0,Año,Mes,Total comercio minorista y vehículos en el dep...,"Comercio, mantenimiento y reparación de vehícu...",NaN,Comercio al por menor,NaN,NaN,NaN,NaN,...,NaN,NaN,Total comercio minorista y vehículos en el dep...,"Comercio, mantenimiento y reparación de vehícu...",NaN,Comercio al por menor,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,Comercio al por menor en establecimientos espe...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Comercio al por menor en establecimientos espe...,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,4511. Vehículos automotores nuevos,"4530- 4541. Partes, piezas (autopartes) y acce...",4711 - 4719 - 4721 -4722 - 4723 - 4724 - 4729 ...,4741 - 4742 - 4751 - 4752 - 4753 - 4754 - 4755...,4761 - 4762 - . 4769 - 4774. Artículos cultura...,4771 - 4772. Prendas de vestir y sus accesorio...,"4773. Productos farmacéuticos, medicinales, od...",...,"4773. Productos farmacéuticos, medicinales, od...",NaN,NaN,4511. Vehículos automotores nuevos,"4530- 4541. Partes, piezas (autopartes) y acce...",4711 - 4719 - 4721 -4722 - 4723 - 4724 - 4729 ...,4741 - 4742 - 4751 - 4752 - 4753 - 4754 - 4755...,4761 - 4762 - . 4769 - 4774. Artículos cultura...,4771 - 4772. Prendas de vestir y sus accesorio...,"4773. Productos farmacéuticos, medicinales, od..."
3,2025,Enero,123.665347,108.626431,134.459701,123.292346,139.810596,118.698212,108.320114,144.791796,...,147.478574,NaN,123.049701,110.331384,138.061059,121.41541,143.688045,144.49922,100.651032,130.540544
4,NaN,Febrero,120.511263,126.738337,130.970545,118.826733,127.039894,94.837225,87.849443,131.151392,...,140.099254,NaN,113.991154,113.478661,135.584995,112.35687,129.718159,108.474957,81.438478,107.829568


In [32]:
# Antes de limpiar, un buen analista siempre diagnostica el estado de los datos.

print("\n--- Diagnóstico Inicial del DataFrame ---")

# .info() nos da un resumen técnico: número de filas, columnas, tipos de datos y valores no nulos.
# Es la "radiografía" de nuestros datos.
print("1. Información General y Tipos de Datos:\n")

df1.info()

# .isnull().sum() nos dice exactamente cuántos valores nulos hay en cada columna.
print("\n2. Conteo de Valores Nulos por Columna:")
print(df1.isnull().sum())

# Creamos una copia del DataFrame para trabajar sobre ella, conservando el original.
# Esta es una práctica fundamental para no perder los datos crudos.
df1_clean = df1.copy()
df1_clean = df1_clean.round(3)


--- Diagnóstico Inicial del DataFrame ---
1. Información General y Tipos de Datos:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 64 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           2 non-null      object 
 1   Unnamed: 1           10 non-null     object 
 2   Antioquia            10 non-null     object 
 3   Unnamed: 3           11 non-null     object 
 4   Unnamed: 4           10 non-null     object 
 5   Unnamed: 5           12 non-null     object 
 6   Unnamed: 6           10 non-null     object 
 7   Unnamed: 7           10 non-null     object 
 8   Unnamed: 8           10 non-null     object 
 9   Unnamed: 9           10 non-null     object 
 10  Unnamed: 10          0 non-null      float64
 11  Atlántico            10 non-null     object 
 12  Unnamed: 12          11 non-null     object 
 13  Unnamed: 13          10 non-null     object 
 14  Unnamed

In [33]:
df1_clean = clean_column_names(df_general)

print("Shape original:", df_general.shape)
display(df1_clean.info())
display(df1_clean.describe(include="all").T)

print("\nDatos faltantes:")
display(summarize_missing(df1_clean).head(15))

# Quitar duplicados
df1_clean = df1_clean.drop_duplicates()

# Columnas constantes
df1_clean, const_cols = drop_constant_columns(df1_clean)
print("\nColumnas constantes eliminadas:", const_cols)

# Intento de detectar columnas de ventas
ventas_cols = [c for c in df1_clean.columns if "venta" in c or "valor" in c or "monto" in c]
print("\nColumnas de ventas detectadas:", ventas_cols)

df1_clean = convert_numeric_columns(df1_clean, ventas_cols)
df1_clean = fill_missing_median(df1_clean)

print("\nShape final tabla1:", df1_clean.shape)
display(df1.head())

def capitalizar_meses_df1_clean(df1_clean):
    meses = {
        'enero': 'Enero','febrero': 'Febrero','marzo': 'Marzo',
        'abril': 'Abril','mayo': 'Mayo','junio': 'Junio',
        'julio': 'Julio','agosto': 'Agosto','septiembre': 'Septiembre',
        'setiembre': 'Septiembre','octubre': 'Octubre',
        'noviembre': 'Noviembre','diciembre': 'Diciembre'
    }
    df1_clean['mes'] = df1_clean['mes'].str.strip().str.lower().map(meses)
    return df1_clean

Shape original: (12, 64)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 64 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   unnamed:_0           2 non-null      object 
 1   unnamed:_1           10 non-null     object 
 2   antioquia            10 non-null     object 
 3   unnamed:_3           11 non-null     object 
 4   unnamed:_4           10 non-null     object 
 5   unnamed:_5           12 non-null     object 
 6   unnamed:_6           10 non-null     object 
 7   unnamed:_7           10 non-null     object 
 8   unnamed:_8           10 non-null     object 
 9   unnamed:_9           10 non-null     object 
 10  unnamed:_10          0 non-null      float64
 11  atlántico            10 non-null     object 
 12  unnamed:_12          11 non-null     object 
 13  unnamed:_13          10 non-null     object 
 14  unnamed:_14          12 non-null     object 
 15  unnamed:_15      

None

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
unnamed:_0,2,2,Año,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
unnamed:_1,10,10,Mes,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
antioquia,10,10,Total comercio minorista y vehículos en el dep...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
unnamed:_3,11,11,"Comercio, mantenimiento y reparación de vehícu...",1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
unnamed:_4,10,10,"4530- 4541. Partes, piezas (autopartes) y acce...",1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
unnamed:_59,12,12,Comercio al por menor,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
unnamed:_60,10,10,4741 - 4742 - 4751 - 4752 - 4753 - 4754 - 4755...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
unnamed:_61,10,10,4761 - 4762 - . 4769 - 4774. Artículos cultura...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
unnamed:_62,10,10,4771 - 4772. Prendas de vestir y sus accesorio...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN



Datos faltantes:


,missing,missing_pct
unnamed:_0,10,83.333333
unnamed:_1,2,16.666667
antioquia,2,16.666667
unnamed:_3,1,8.333333
unnamed:_4,2,16.666667
unnamed:_5,0,0.000000
unnamed:_6,2,16.666667
unnamed:_7,2,16.666667
unnamed:_8,2,16.666667
unnamed:_9,2,16.666667



Columnas constantes eliminadas: ['unnamed:_10', 'unnamed:_19', 'unnamed:_28', 'unnamed:_37', 'unnamed:_46', 'unnamed:_55']

Columnas de ventas detectadas: []

Shape final tabla1: (12, 58)


,Unnamed: 0,Unnamed: 1,Antioquia,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 54,Unnamed: 55,Otros departamentos,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63
0,Año,Mes,Total comercio minorista y vehículos en el dep...,"Comercio, mantenimiento y reparación de vehícu...",NaN,Comercio al por menor,NaN,NaN,NaN,NaN,...,NaN,NaN,Total comercio minorista y vehículos en el dep...,"Comercio, mantenimiento y reparación de vehícu...",NaN,Comercio al por menor,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,Comercio al por menor en establecimientos espe...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Comercio al por menor en establecimientos espe...,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,4511. Vehículos automotores nuevos,"4530- 4541. Partes, piezas (autopartes) y acce...",4711 - 4719 - 4721 -4722 - 4723 - 4724 - 4729 ...,4741 - 4742 - 4751 - 4752 - 4753 - 4754 - 4755...,4761 - 4762 - . 4769 - 4774. Artículos cultura...,4771 - 4772. Prendas de vestir y sus accesorio...,"4773. Productos farmacéuticos, medicinales, od...",...,"4773. Productos farmacéuticos, medicinales, od...",NaN,NaN,4511. Vehículos automotores nuevos,"4530- 4541. Partes, piezas (autopartes) y acce...",4711 - 4719 - 4721 -4722 - 4723 - 4724 - 4729 ...,4741 - 4742 - 4751 - 4752 - 4753 - 4754 - 4755...,4761 - 4762 - . 4769 - 4774. Artículos cultura...,4771 - 4772. Prendas de vestir y sus accesorio...,"4773. Productos farmacéuticos, medicinales, od..."
3,2025,Enero,123.665347,108.626431,134.459701,123.292346,139.810596,118.698212,108.320114,144.791796,...,147.478574,NaN,123.049701,110.331384,138.061059,121.41541,143.688045,144.49922,100.651032,130.540544
4,NaN,Febrero,120.511263,126.738337,130.970545,118.826733,127.039894,94.837225,87.849443,131.151392,...,140.099254,NaN,113.991154,113.478661,135.584995,112.35687,129.718159,108.474957,81.438478,107.829568


##### Base de datos 2: Informativos Comercio de Vehículos - Por departamentos (EMC)

In [34]:
file_path2 = 'tabla2_emc_vehiculos.xlsx'

# Se lee el archivo Excel y se carga en un DataFrame de Pandas.
# El bloque 'try-except' es una buena práctica para manejar errores si el archivo no existe.
try:
    df2 = pd.read_excel(file_path2)
    print("✅ Archivo 'tabla2_emc_vehiculos.xlsx' cargado exitosamente.")
except FileNotFoundError:
    print(f"❌ Error: No se encontró el archivo '{file_path2}'. Asegúrate de que esté en la misma carpeta.")

# Mostramos las primeras 5 filas para tener una idea inicial de los datos.
print("\n--- Vista Previa de los Datos Crudos ('Sucios') ---")
df2.head()

✅ Archivo 'tabla2_emc_vehiculos.xlsx' cargado exitosamente.

--- Vista Previa de los Datos Crudos ('Sucios') ---


,Tipo de vehículo,Unnamed: 1,Vehículos de origen nacional + importado,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,NaN,NaN,Automóviles particulares,NaN,Camperos y camionetas,NaN,Vehículos de transp. público,NaN,Vehículos de carga,NaN,Motocicletasa,NaN
1,NaN,NaN,Unidades vendidas,Valor ventas,Unidades vendidas,Valor ventas,Unidades vendidas,Valor ventas,Unidades vendidas,Valor ventas,Unidades vendidas,Valor ventas
2,2025.0,Enero,3405,241201385,9471,1181596926,553,82856022,1068,247300066,76268,531886954
3,NaN,Febrero,4531,307182298,8910,1071946799,487,64792361,1296,233869642,83739,596627061
4,NaN,Marzo,4712,321924982,10712,1262303106,492,60273624,1281,198738588,85762,604323891


In [35]:
# Antes de limpiar, un buen analista siempre diagnostica el estado de los datos.

print("\n--- Diagnóstico Inicial del DataFrame ---")

# .info() nos da un resumen técnico: número de filas, columnas, tipos de datos y valores no nulos.
# Es la "radiografía" de nuestros datos.
print("1. Información General y Tipos de Datos:\n")

df2.info()

# .isnull().sum() nos dice exactamente cuántos valores nulos hay en cada columna.
print("\n2. Conteo de Valores Nulos por Columna:")
print(df2.isnull().sum())

# Creamos una copia del DataFrame para trabajar sobre ella, conservando el original.
# Esta es una práctica fundamental para no perder los datos crudos.
df2_clean = df2.copy()
df2_clean = df2_clean.round(3)


# Retirar mayusculas para facilitar el acceso al proceso de finaniciación
def capitalizar_meses_df2_clean(df2_clean):
    meses = {
        'enero': 'Enero','febrero': 'Febrero','marzo': 'Marzo',
        'abril': 'Abril','mayo': 'Mayo','junio': 'Junio',
        'julio': 'Julio','agosto': 'Agosto','septiembre': 'Septiembre',
        'setiembre': 'Septiembre','octubre': 'Octubre',
        'noviembre': 'Noviembre','diciembre': 'Diciembre'
    }


--- Diagnóstico Inicial del DataFrame ---
1. Información General y Tipos de Datos:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 12 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Tipo de vehículo                          1 non-null      float64
 1   Unnamed: 1                                9 non-null      object 
 2   Vehículos de origen nacional + importado  11 non-null     object 
 3   Unnamed: 3                                10 non-null     object 
 4   Unnamed: 4                                11 non-null     object 
 5   Unnamed: 5                                10 non-null     object 
 6   Unnamed: 6                                11 non-null     object 
 7   Unnamed: 7                                10 non-null     object 
 8   Unnamed: 8                                11 non-null     object 
 9   Unnamed: 9                

In [36]:
df2_clean = clean_column_names(df_vehiculos)

print("Shape original:", df_vehiculos.shape)
display(df2_clean.info())
display(df2_clean.describe(include="all").T)
display(summarize_missing(df2_clean).head(20))

# Convertir fechas si existen
for c in df2_clean.columns:
    if "fecha" in c or "anio" in c:
        df2_clean[c] = pd.to_datetime(df2_clean[c], errors="coerce")

# Normalización de localización
loc_cols = [c for c in df2_clean.columns if "depart" in c or "ciudad" in c]
for c in loc_cols:
    df2_clean[c] = df2_clean[c].astype(str).str.strip().str.title()

# Columnas numéricas importantes
num_cols = [c for c in df2_clean.columns if "venta" in c or "cantidad" in c or "valor" in c]
df2_clean = convert_numeric_columns(df2_clean, num_cols)

# Llenar NA
df2_clean = fill_missing_median(df2_clean)

# Quitar duplicados
df2_clean = df2_clean.drop_duplicates()

print("\nShape final tabla2:", df2_clean.shape)
display(df2_clean.head())


Shape original: (11, 12)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 12 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   tipo_de_vehículo                          1 non-null      float64
 1   unnamed:_1                                9 non-null      object 
 2   vehículos_de_origen_nacional_+_importado  11 non-null     object 
 3   unnamed:_3                                10 non-null     object 
 4   unnamed:_4                                11 non-null     object 
 5   unnamed:_5                                10 non-null     object 
 6   unnamed:_6                                11 non-null     object 
 7   unnamed:_7                                10 non-null     object 
 8   unnamed:_8                                11 non-null     object 
 9   unnamed:_9                                10 non-null     object 
 10  unnamed:_10    

None

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
tipo_de_vehículo,1.0,NaN,NaN,NaN,2025.0,NaN,2025.0,2025.0,2025.0,2025.0,2025.0
unnamed:_1,9,9,Enero,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
vehículos_de_origen_nacional_+_importado,11,11,Automóviles particulares,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
unnamed:_3,10,10,Valor ventas,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
unnamed:_4,11,11,Camperos y camionetas,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
unnamed:_5,10,10,Valor ventas,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
unnamed:_6,11,11,Vehículos de transp. público,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
unnamed:_7,10,10,Valor ventas,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
unnamed:_8,11,11,Vehículos de carga,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
unnamed:_9,10,10,Valor ventas,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,missing,missing_pct
tipo_de_vehículo,10,90.909091
unnamed:_1,2,18.181818
vehículos_de_origen_nacional_+_importado,0,0.000000
unnamed:_3,1,9.090909
unnamed:_4,0,0.000000
unnamed:_5,1,9.090909
unnamed:_6,0,0.000000
unnamed:_7,1,9.090909
unnamed:_8,0,0.000000
unnamed:_9,1,9.090909



Shape final tabla2: (11, 12)


,tipo_de_vehículo,unnamed:_1,vehículos_de_origen_nacional_+_importado,unnamed:_3,unnamed:_4,unnamed:_5,unnamed:_6,unnamed:_7,unnamed:_8,unnamed:_9,unnamed:_10,unnamed:_11
0,2025.0,NaN,Automóviles particulares,NaN,Camperos y camionetas,NaN,Vehículos de transp. público,NaN,Vehículos de carga,NaN,Motocicletasa,NaN
1,2025.0,NaN,Unidades vendidas,Valor ventas,Unidades vendidas,Valor ventas,Unidades vendidas,Valor ventas,Unidades vendidas,Valor ventas,Unidades vendidas,Valor ventas
2,2025.0,Enero,3405,241201385,9471,1181596926,553,82856022,1068,247300066,76268,531886954
3,2025.0,Febrero,4531,307182298,8910,1071946799,487,64792361,1296,233869642,83739,596627061
4,2025.0,Marzo,4712,321924982,10712,1262303106,492,60273624,1281,198738588,85762,604323891


##### Base de datos 3: PERSONAS (Demográfico)

In [37]:
file_path3 = 'tabla3_demografia_censo2018.xlsx'

# Se lee el archivo Excel y se carga en un DataFrame de Pandas.
# El bloque 'try-except' es una buena práctica para manejar errores si el archivo no existe.
try:
    df3 = pd.read_excel(file_path3)
    print("✅ Archivo 'tabla3_demografia_censo2018.xlsx' cargado exitosamente.")
except FileNotFoundError:
    print(f"❌ Error: No se encontró el archivo '{file_path3}'. Asegúrate de que esté en la misma carpeta.")

# Mostramos las primeras 5 filas para tener una idea inicial de los datos.
print("\n--- Vista Previa de los Datos Crudos ('Sucios') ---")
df3.head()

✅ Archivo 'tabla3_demografia_censo2018.xlsx' cargado exitosamente.

--- Vista Previa de los Datos Crudos ('Sucios') ---


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,2018,NaN,NaN,NaN,NaN,NaN
1,"Grupos de edad, departamento y Áreas (Total, ...",NaN,NaN,Total,Sexo,NaN
2,NaN,NaN,NaN,NaN,Hombre,Mujer
3,Total Nacional,Total,Total,44164417,21570493,22593924
4,NaN,NaN,0 a 4,3037781,1555605,1482176


In [38]:
# Antes de limpiar, un buen analista siempre diagnostica el estado de los datos.

print("\n--- Diagnóstico Inicial del DataFrame ---")

# .info() nos da un resumen técnico: número de filas, columnas, tipos de datos y valores no nulos.
# Es la "radiografía" de nuestros datos.
print("1. Información General y Tipos de Datos:\n")

df3.info()

# .isnull().sum() nos dice exactamente cuántos valores nulos hay en cada columna.
print("\n2. Conteo de Valores Nulos por Columna:")
print(df3.isnull().sum())

# Creamos una copia del DataFrame para trabajar sobre ella, conservando el original.
# Esta es una práctica fundamental para no perder los datos crudos.
df3_clean = df3.copy()
df3_clean = df3_clean.round(3)


--- Diagnóstico Inicial del DataFrame ---
1. Información General y Tipos de Datos:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2587 entries, 0 to 2586
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  36 non-null     object
 1   Unnamed: 1  136 non-null    object
 2   Unnamed: 2  2584 non-null   object
 3   Unnamed: 3  2585 non-null   object
 4   Unnamed: 4  2586 non-null   object
 5   Unnamed: 5  2585 non-null   object
dtypes: object(6)
memory usage: 121.4+ KB

2. Conteo de Valores Nulos por Columna:
Unnamed: 0    2551
Unnamed: 1    2451
Unnamed: 2       3
Unnamed: 3       2
Unnamed: 4       1
Unnamed: 5       2
dtype: int64


In [39]:
df3_clean = clean_column_names(df_demografia)

print("Shape original:", df_demografia.shape)
display(df3_clean.info())
display(df3_clean.describe(include="all").T)
display(summarize_missing(df3_clean).head(20))

# Convertir columnas a numéricas si son números disfrazados de texto
cols_to_convert = []
for c in df3_clean.columns:
    sample = df3_clean[c].dropna().astype(str).head(50)
    if sample.str.replace(",", "").str.replace(".", "").str.isnumeric().mean() > 0.4:
        cols_to_convert.append(c)

print("\nColumnas a convertir:", cols_to_convert)
df3_clean = convert_numeric_columns(df3_clean, cols_to_convert)

# Estandarizar ubicación
loc_cols = [c for c in df3_clean.columns if "depart" in c or "municip" in c]
for c in loc_cols:
    df3[c] = df3[c].astype(str).str.strip().str.title()

# Retirar mayusculas para facilitar el acceso al proceso de finaniciación
def capitalizar_meses_df3(df3):
    meses = {
        'enero': 'Enero','febrero': 'Febrero','marzo': 'Marzo',
        'abril': 'Abril','mayo': 'Mayo','junio': 'Junio',
        'julio': 'Julio','agosto': 'Agosto','septiembre': 'Septiembre',
        'setiembre': 'Septiembre','octubre': 'Octubre',
        'noviembre': 'Noviembre','diciembre': 'Diciembre'
    }
    df3['periodo'] = df3['periodo'].str.strip().str.lower().map(meses)
    return df3

# Llenar NA
df3_clean = fill_missing_median(df3_clean)

print("\nShape final tabla3:", df3_clean.shape)
display(df3_clean.head())



Shape original: (2587, 6)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2587 entries, 0 to 2586
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   unnamed:_0  36 non-null     object
 1   unnamed:_1  136 non-null    object
 2   unnamed:_2  2584 non-null   object
 3   unnamed:_3  2585 non-null   object
 4   unnamed:_4  2586 non-null   object
 5   unnamed:_5  2585 non-null   object
dtypes: object(6)
memory usage: 121.4+ KB


None

,count,unique,top,freq
unnamed:_0,36,36,2018,1
unnamed:_1,136,4,Total,34
unnamed:_2,2584,19,Total,136
unnamed:_3,2585,2448,1081,3
unnamed:_4,2586,2345,22,6
unnamed:_5,2585,2345,10,6


,missing,missing_pct
unnamed:_0,2551,98.608427
unnamed:_1,2451,94.742945
unnamed:_2,3,0.115964
unnamed:_3,2,0.077310
unnamed:_4,1,0.038655
unnamed:_5,2,0.077310



Columnas a convertir: ['unnamed:_3', 'unnamed:_4', 'unnamed:_5']

Shape final tabla3: (2587, 6)


,unnamed:_0,unnamed:_1,unnamed:_2,unnamed:_3,unnamed:_4,unnamed:_5
0,2018,NaN,NaN,12270.5,6193.5,6259.5
1,"Grupos de edad, departamento y Áreas (Total, ...",NaN,NaN,12270.5,6193.5,6259.5
2,NaN,NaN,NaN,12270.5,6193.5,6259.5
3,Total Nacional,Total,Total,44164417.0,21570493.0,22593924.0
4,NaN,NaN,0 a 4,3037781.0,1555605.0,1482176.0


In [49]:
# Exportar el Resultado

print("\n--- Iniciando Fase de Limpieza Final y Preparación para Exportación ---")

# Creamos una copia final para aplicar las últimas transformaciones.
# Es una buena práctica para no modificar el DataFrame anterior (df_clean).
df1_final = df1_clean.copy()
df2_final = df2_clean.copy()
df3_final = df3_clean.copy()

import os

# Exportar a Excel
output_filename = 'EMC_General_Limpio.xlsx'
df1_final.to_excel(output_filename, index=False)
output_filename2 = 'EMC_Vehiculos_Limpio.xlsx'
df2_final.to_excel(output_filename2, index=False)
output_filename3 = 'Demografia_Censo2018_Limpio.xlsx'
df3_final.to_excel(output_filename3, index=False)

print("✅ Archivos exportados exitosamente en formato Excel (.xlsx).")


--- Iniciando Fase de Limpieza Final y Preparación para Exportación ---
✅ Archivos exportados exitosamente en formato Excel (.xlsx).
